# Segmentation
In this module we will extend the basic idea of convolutional neural networks from classifying an image to generating an alterbative image. In the context of remote sensing, this means ingesting one or more remote sensing images and generating a raster or map.

This type of network is called an 'encoder-decoder'

TODO - more here

## Preliminaries
First we'll import the libraries we need.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from skimage import io, img_as_ubyte
from skimage.transform import rescale

import torch # Loads the CUDA libraries for tensorflow

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam

#from keras.callbacks import ModelCheckpoint, LearningRateScheduler
#from keras import backend as keras
#from keras import utils as ku

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import glob

import random

# Keras utilities
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#from tensorflow.keras.optimizers import Adam

Now we'll import some utilities for visualisation etc. If you want to take a look at this code, uncomment the second line below before running it.

In [ ]:
from files import utils
# %load files/utils

## A simple model
We'll start with the simplest encoder-decoder network:

<img src="files/encoder_decoder.png" width="500"/>

- the encoder consists of a single convolutional layer that looks for certain patterns and uses them to transform the input imagery into feature maps highlighting those salient patterns in the image
- the decoder network also consists of just a single convolutional layer. This layer has one 'filter' per class, and this filter projects (collapses) the all the feature map outputs for each individual pixel into a single pixel value. In other words, it learns to assign a weight to each of the feature maps for each class.


In [ ]:
def encoder_decoder(num_classes, input_size):
    num_filters = 64 # 16 # also 4, 16
    filter_size = 3 # also 4,5

    kernel_init = initializers.RandomUniform(minval=-0.01, maxval=0.01) # For visualisation

    # Preprocess the input
    inputs = layers.Input(input_size, name='Input')
    rescale = layers.Rescaling(1.0 / 255, name='Rescale')(inputs) # Rescale to 0..1

    # Encoder
    conv1 = layers.Conv2D(num_filters, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init, name='Encoder')(rescale)

    # Decoder
    out = layers.Conv2D(num_classes, 1, activation="softmax", padding="same", kernel_initializer=kernel_init, name='Decoder')(conv1)

    model = Model(inputs, out)
    return model

Let's build our model.

In [ ]:
num_classes = 3 # (0=background; 1=nought; 2=cross)
image_size = 32
input_size = (image_size,image_size,3)
# learning_rate = 1e-4 # Our go-to learning rate
learning_rate = 0.01 # Fast learning

model = encoder_decoder(num_classes, input_size)
model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])
model.summary(line_length=80)


let's take a look at the initial model weights.

In [ ]:
utils.report_weights(model)

## Training
Lets try out our hyper-simple model on some real data. We'll use noughts and crosses again, but this time we want the network to generate a mask that assigns the value 1 to noughts and 2 to crosses. Here's an example input and the expected output:

<table><tr>
        <td><img src="files/seg_data/seg_clean/train/images/65.png" width="200"/></td>
        <td><img src="files/seg_data/seg_clean/train/visual/65.png_visual.png" width="200"/></td>
</tr></table>


Training is slightly more complex this time. First, we'll need a couple of data generators that prepare the input images and label masks for ingestion.

In [ ]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,target_size):
    # Yields batches of (image,mask) of batch_size

    image_path = os.path.join(train_path, image_folder)
    mask_path = os.path.join(train_path, mask_folder)
    
    i = 0
    n = 0
    (width, height) = target_size
    
    while True: # The caller will decide when to stop
        # Generate a randomised list of (image, mask) filenames
        files = list(zip(sorted(list(glob.glob(image_path + '/*'))), sorted(list(glob.glob(mask_path + '/*')))))
        random.shuffle(files)
        
        for (filename, mask_filename) in files:
            mask = io.imread(mask_filename)
            if np.sum(mask) == 0: # ignore images with blank masks
                pass
            else:
                img = io.imread(filename)
                if i == 0:
                    images = np.zeros((batch_size, img.shape[0], img.shape[1], img.shape[2]))
                    masks = np.zeros((batch_size, mask.shape[0], mask.shape[1], 1))
    
                images[i] = img
                masks[i] = np.expand_dims(mask[:,:,0],-1)
                
                i += 1
                if i == batch_size:
                    i = 0
                    # images, masks = adjustData(images,masks,flag_multi_class,num_class) # No need
                    if np.max(masks) > 0:
                        yield (images, masks)

def testGenerator(test_path, target_size, offset=0, num_files=0):
    (width, height) = target_size

    files = sorted(glob.glob(test_path + '/*'))

    if num_files == 0:
        num_files = len(files)
    
    i = 0
    while i < num_files:
        filename = files[i+offset]
        img = io.imread(filename)
        # img = np.reshape(img,(1,) + img.shape) # Make into a batch of size 1
        img = np.expand_dims(img,0) # Make into a batch of size 1
        i += 1
        yield img

Now we'll train our model.

In [ ]:
# Train the model

IMAGES_DIR = 'images/'
LABELS_DIR = 'labels/'

data_dir = "files/seg_data/seg_clean/"
train_dir = data_dir + "/train/"
# train_dir = data_dir + "/train_debug/" # One image only - should easily learn it to 100%
test_dir = data_dir + "/test/" # All three bands contain the character
out_dir = data_dir + "out/"

#STEPS_PER_EPOCH = 50 # Entire training set
TRAIN_BATCH_SIZE = 2
STEPS_PER_EPOCH = 50 # Entire training set
# EPOCHS = 1 # Quick test
# EPOCHS = 5 # Maximal for Unet64? No - it's a mess!
EPOCHS = 20
# EPOCHS = 100
# EPOCHS = 10 # Learning anything???
       
train_gen = trainGenerator(TRAIN_BATCH_SIZE, train_dir, IMAGES_DIR, LABELS_DIR, target_size=(image_size,image_size))
val_gen = trainGenerator(TRAIN_BATCH_SIZE, test_dir, IMAGES_DIR, LABELS_DIR, target_size=(image_size,image_size))

# model.fit_generator(train_gen,steps_per_epoch=STEPS_PER_EPOCH,epochs=num_epochs, callbacks=[model_checkpoint], verbose=VERBOSITY)
#model.fit_generator(train_gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, 
#                    validation_data = val_gen, validation_steps=STEPS_PER_EPOCH)

model.fit(train_gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, validation_data = val_gen, validation_steps=STEPS_PER_EPOCH)

print("\n*** TRAINING COMPLETE ***")

## Testing
For segmentation we are interested in the model's ability to generate the desired output. First, let's test the model on an image and see how the result compares to our expectations.

In [ ]:
test_file = '65.png'
test_image = test_dir + 'images/' + test_file
test_mask = test_dir + 'labels/' + test_file

img = io.imread(test_image)
img = np.expand_dims(img,0) # make into a batch of size 1
#results = list(model.predict_generator(testGene, 1, verbose=1))
result = list(model.predict(img, 1, verbose=0))[0]
# Convert the result to a mask
mask = np.argmax(result, axis = 2).astype(np.uint8) # Finds the index of the highest class probability for each (x,y)

# Plot the original image, the result, and the probabilities
# TODO move to utils ("plot_images")
figure, axis = plt.subplots(1,4)
[axis[i].set_axis_off() for i in range (0,4)]
[axis[i].figure.set_figwidth(10) for i in range (0,4)]
axis[0].imshow(img[0])
axis[1].imshow(mask)
axis[2].imshow(result[:,:,1], cmap='gray') # class = 1 (nought)
axis[3].imshow(result[:,:,2], cmap='gray') # class = 2 (cross)
plt.show()

## Visualising performance
In this simple example it's not hard to see where the model was correct/wrong, 
but this obviously gets a lot more difficult as the complexity of the imagery and/or labelling rises.

One way to solve this is to generate a comparison of the desired versus generated results. In the visualisation below:
- Blue = true negative (both truth and prediction are class 0/background)
- White = true positive (both truth and prediction are the same non-zero class)
- Yellow = wrong class (truth and prediction are *different* non-zero class)
- Red = false negative (truth is non-zero, predicted is zero)
- Green = false positive (truth is zero, predicted is non-zero)

In [ ]:
im_truth = io.imread(test_mask)
error_shape = im_truth.shape
if len(error_shape) == 2:
    error_shape = error_shape + (3,)
im_error = np.zeros(error_shape)

# Grab the first channel only if RGB masks
if len(im_truth.shape) == 3: im_truth = im_truth[:,:,0]
if len(mask.shape) == 3: im_predicted = im_predicted[:,:,0]

im_error[:,:,0] = (im_truth > 0) # Red: truth is non-zero
im_error[:,:,1] = (mask > 0) # Green: prediction is non-zero
im_error[:,:,2] = (mask == im_truth) # Blue: prediction = truth

plt.imshow(im_error)

In [ ]:
utils.report_outputs(model, test_image, image_size)

## Measuring performance
There are several ways to objectively measure (pixel) performance, including:
- Overall accuracy: number of correct pixels
- IOU (intersection over union): percentage of (truth U prediction) pixels that overlap, for each (non-zero) class
- Confusion: shows the patterns of correct and incorrect results by class
- Precision: percentage of truth pixels of a given class that were correctly classified (versus false negatives)
- Recall: percentage of predictions for a given class that were correctly classified (versus false positives)

In a mapping context, we may also be interested in measuring accuracy per-polygon basis; IOU is particularly useful in this context.

## A more complex network
The network we just trained is extremely simple - it is just one layer "deep". We will increase its abstraction ability by adding more layers.
Take a look at the network definition below.

<img src="files/unet_lite.png" width="500"/>


In [ ]:
def unet_lite(num_classes, input_size):
    num_filters = 8 #64 # 16 # also 4, 16
    filter_size = 3 # also 4,5
    # num_hidden = 64 # also 16?)

    kernel_init = initializers.RandomUniform(minval=-0.01, maxval=0.01) # For visualisation

    # Preprocess the input
    inputs = layers.Input(input_size, name='Inputs')
    rescale = layers.Rescaling(1.0 / 255, name='Rescale')(inputs) # Rescale to 0..1

    # Encoder
    conv1 = layers.Conv2D(num_filters, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init, name='Encoder-1')(rescale)
    down1 = layers.MaxPooling2D(name='MaxPool')(conv1)

    head = layers.Conv2D(num_filters * 2, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init, name='Encoder-2')(down1)

    # Decoder
    upsample1 = layers.UpSampling2D(size=(2, 2),name='Upsample')(head)
    up1 = layers.Conv2D(num_filters * 2, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init, name = 'Decoder-1')(upsample1)

    merge1 = layers.concatenate([conv1,up1], axis=3, name='Merge') # Merge the encoder and decoder outputs with the same scale

    out = layers.Conv2D(num_classes, 1, activation="softmax", padding="same", kernel_initializer=kernel_init, name='Decoder-2')(merge1)
   
    model = Model(inputs, out, name="Unet-lite")
    return model

model = unet_lite(num_classes, input_size)
model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])
model.summary()

In [ ]:
model.fit(train_gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, validation_data = val_gen, validation_steps=STEPS_PER_EPOCH)

print("\n*** TRAINING COMPLETE ***")

In [ ]:
utils.report_outputs(model, test_image, image_size)

## Unet
The original Unet is exactly like our light version except it is several layers deep. Let's see how it performs.

In [ ]:
def unet(num_classes, input_size):
    width = 16 # originally 64
    
    inputs = Input(input_size)
    rescale = layers.Rescaling(1.0 / 255)(inputs) # Rescale to 0..1 - BIM
    conv1 = Conv2D(width, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(rescale) # (inputs)
    conv1 = Conv2D(width, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)   
    conv2 = Conv2D(width*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(width*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)    
    conv3 = Conv2D(width*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(width*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)

    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)    
    conv4 = Conv2D(width*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(width*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)

    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    conv5 = Conv2D(width*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(width*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(width*8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(width*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(width*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(width*4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(width*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(width*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(width*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(width*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(width*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(width, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    
    conv9 = Conv2D(width, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(width, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2*num_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(num_classes, 1, activation = 'softmax', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        
    model = Model(inputs, conv10, name='Unet64')

    return model

model = unet(num_classes, input_size)
model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])
model.summary()

In [ ]:
model.fit(train_gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, validation_data = val_gen, validation_steps=STEPS_PER_EPOCH)

print("\n*** TRAINING COMPLETE ***")

In [ ]:
utils.report_outputs(model, test_image, image_size)